In [3]:
from mysql.connector import (connection)
import mysql

cnx = connection.MySQLConnection(user='root', password='root',
                                 host='127.0.0.1',
                                 database='test')
# cnx.close()

In [14]:
from mysql.connector import Error, errorcode

# ref: https://www.mysqltutorial.org/mysql-json/
def create(table_name, table_description):
    try:
        cursor = cnx.cursor()
        print("Creating table {} ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    finally:
        cursor.close()

create('events', '''CREATE TABLE events( 
  id int auto_increment primary key, 
  event_name varchar(255), 
  visitor varchar(255), 
  properties json, 
  browser json
);''')

Creating table events already exists.


In [9]:
add_rec='''INSERT INTO events(event_name, visitor,properties, browser) 
VALUES (
  'pageview', 
   '1',
   '{ "page": "/" }',
   '{ "name": "Safari", "os": "Mac", "resolution": { "x": 1920, "y": 1080 } }'
),
('pageview', 
  '2',
  '{ "page": "/contact" }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 2560, "y": 1600 } }'
),
(
  'pageview', 
  '1',
  '{ "page": "/products" }',
  '{ "name": "Safari", "os": "Mac", "resolution": { "x": 1920, "y": 1080 } }'
),
(
  'purchase', 
   '3',
  '{ "amount": 200 }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 1600, "y": 900 } }'
),
(
  'purchase', 
   '4',
  '{ "amount": 150 }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 1280, "y": 800 } }'
),
(
  'purchase', 
  '4',
  '{ "amount": 500 }',
  '{ "name": "Chrome", "os": "Windows", "resolution": { "x": 1680, "y": 1050 } }'
);
'''
cursor = cnx.cursor()
cursor.execute(add_rec)
cnx.commit()
cursor.close()

True

In [11]:
# query='''SELECT id, browser->'$.name' browser
# FROM events;
# '''
# To remove the quote marks, you use the inline path operator (->>)
query='''SELECT id, browser->>'$.name' browser
FROM events;
'''
cursor = cnx.cursor()
cursor.execute(query)

for (id, browser) in cursor:
    print(f"rec id:{id}, browser:{browser}")

cursor.close()

rec id:1, browser:Safari
rec id:2, browser:Firefox
rec id:3, browser:Safari
rec id:4, browser:Firefox
rec id:5, browser:Firefox
rec id:6, browser:Chrome


True

In [12]:
query='''SELECT browser->>'$.name' browser, 
      count(browser)
FROM events
GROUP BY browser->>'$.name';
'''
cursor = cnx.cursor()
cursor.execute(query)

for (browser,count) in cursor:
    print(f"rec {browser}, {count}")

cursor.close()

rec Safari, 2
rec Firefox, 3
rec Chrome, 1


True

In [19]:
# To calculate the total revenue by the visitor, you use the following query:
# paras={'amount':201}
paras={'amount':0}
query='''SELECT visitor, SUM(properties->>'$.amount') revenue
FROM events
WHERE properties->>'$.amount' > %(amount)s
GROUP BY visitor;
'''
cursor = cnx.cursor()
cursor.execute(query, paras)

for (visitor,revenue) in cursor:
    print(f"rec {visitor}, {revenue}")

cursor.close()

rec 3, 200.0
rec 4, 650.0


True

In [16]:
?cursor.execute